In [12]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

https://pennylane.ai/qml/demos/tutorial_qubit_rotation.html

Basic tutorial: qubit rotation {#qubit_rotation}
==============================

::: {.meta}
:property=\"og:description\": To see how PennyLane allows the easy
construction and optimization of quantum functions, let\'s consider the
\'hello world\' of QML: qubit rotation. :property=\"og:image\":
<https://pennylane.ai/qml/_images/bloch.png>
:::

::: {.related}
plugins\_hybrid Plugins and hybrid computation
tutorial\_gaussian\_transformation Gaussian transformation
tutorial\_state\_preparation Training a quantum circuit with PyTorch
:::

*Author: Josh Izaac --- Posted: 11 October 2019. Last updated: 19
January 2021.*

To see how PennyLane allows the easy construction and optimization of
quantum functions, let\'s consider the simple case of **qubit rotation**
the PennyLane version of the \'Hello, world!\' example.

The task at hand is to optimize two rotation gates in order to flip a
single qubit from state $\left|0\right\rangle$ to state
$\left|1\right\rangle$.

The quantum circuit
-------------------

In the qubit rotation example, we wish to implement the following
quantum circuit:

![](../demonstrations/qubit_rotation/rotation_circuit.png){.align-center
width="40.0%"}

Breaking this down step-by-step, we first start with a qubit in the
ground state $|0\rangle = \begin{bmatrix}1 & 0 \end{bmatrix}^T$, and
rotate it around the x-axis by applying the gate

$$\begin{aligned}
R_x(\phi_1) = e^{-i \phi_1 \sigma_x /2} =
\begin{bmatrix} \cos \frac{\phi_1}{2} &  -i \sin \frac{\phi_1}{2} \\
-i \sin \frac{\phi_1}{2} &  \cos \frac{\phi_1}{2}
\end{bmatrix},
\end{aligned}$$

and then around the y-axis via the gate

$$\begin{aligned}
R_y(\phi_2) = e^{-i \phi_2 \sigma_y/2} =
\begin{bmatrix} \cos \frac{\phi_2}{2} &  - \sin \frac{\phi_2}{2} \\
\sin \frac{\phi_2}{2} &  \cos \frac{\phi_2}{2}
\end{bmatrix}.
\end{aligned}$$

After these operations the qubit is now in the state

$$| \psi \rangle = R_y(\phi_2) R_x(\phi_1) | 0 \rangle.$$

Finally, we measure the expectation value
$\langle \psi \mid \sigma_z \mid \psi \rangle$ of the Pauli-Z operator

$$\begin{aligned}
\sigma_z =
\begin{bmatrix} 1 &  0 \\
0 & -1
\end{bmatrix}.
\end{aligned}$$

Using the above to calculate the exact expectation value, we find that

$$\langle \psi \mid \sigma_z \mid \psi \rangle
= \langle 0 \mid R_x(\phi_1)^\dagger R_y(\phi_2)^\dagger \sigma_z  R_y(\phi_2) R_x(\phi_1) \mid 0 \rangle
= \cos(\phi_1)\cos(\phi_2).$$

Depending on the circuit parameters $\phi_1$ and $\phi_2$, the output
expectation lies between $1$ (if
$\left|\psi\right\rangle = \left|0\right\rangle$) and $-1$ (if
$\left|\psi\right\rangle = \left|1\right\rangle$).


In [2]:
import pennylane as qml
from pennylane import numpy as np

In [3]:
dev1 = qml.device("default.qubit", wires=1)

In [4]:
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))

In [5]:
@qml.qnode(dev1, interface="autograd")
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))

Thus, our `circuit()` quantum function is now a
`~.pennylane.QNode`{.interpreted-text role="class"}, which will run on
device `dev1` every time it is evaluated.

To evaluate, we simply call the function with some appropriate numerical
inputs:


In [6]:
print(circuit([0.54, 0.12]))

0.8515405859048367


Calculating quantum gradients
=============================

The gradient of the function `circuit`, encapsulated within the `QNode`,
can be evaluated by utilizing the same quantum device (`dev1`) that we
used to evaluate the function itself.

PennyLane incorporates both analytic differentiation, as well as
numerical methods (such as the method of finite differences). Both of
these are done automatically.

We can differentiate by using the built-in
`~.pennylane.grad`{.interpreted-text role="func"} function. This returns
another function, representing the gradient (i.e., the vector of partial
derivatives) of `circuit`. The gradient can be evaluated in the same way
as the original function:


In [7]:
dcircuit = qml.grad(circuit, argnum=0)

The function `~.pennylane.grad`{.interpreted-text role="func"} itself
**returns a function**, representing the derivative of the QNode with
respect to the argument specified in `argnum`. In this case, the
function `circuit` takes one argument (`params`), so we specify
`argnum=0`. Because the argument has two elements, the returned gradient
is two-dimensional. We can then evaluate this gradient function at any
point in the parameter space.


In [8]:
print(dcircuit([0.54, 0.12]))

[array(-0.51043865), array(-0.1026782)]


Optimization
============
In this case, our desired outcome is a Pauli-Z expectation value of
$-1$. Since we know that the Pauli-Z expectation is bound between
$[-1, 1]$, we can define our cost directly as the output of the QNode:


In [9]:
def cost(x):
    return circuit(x)

To begin our optimization, let\'s choose small initial values of
$\phi_1$ and $\phi_2$:


In [10]:
init_params = np.array([0.011, 0.012], requires_grad=True)
print(cost(init_params))

0.9998675058299389


We can see that, for these initial parameter values, the cost function
is close to $1$.

Finally, we use an optimizer to update the circuit parameters for 100
steps. We can use the built-in
`~.pennylane.GradientDescentOptimizer`{.interpreted-text role="class"}
class:


In [11]:
# initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# set the number of steps
steps = 100
# set the initial parameter values
params = init_params

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.9961778
Cost after step    10:  0.8974944
Cost after step    15:  0.1440490
Cost after step    20: -0.1536720
Cost after step    25: -0.9152496
Cost after step    30: -0.9994046
Cost after step    35: -0.9999964
Cost after step    40: -1.0000000
Cost after step    45: -1.0000000
Cost after step    50: -1.0000000
Cost after step    55: -1.0000000
Cost after step    60: -1.0000000
Cost after step    65: -1.0000000
Cost after step    70: -1.0000000
Cost after step    75: -1.0000000
Cost after step    80: -1.0000000
Cost after step    85: -1.0000000
Cost after step    90: -1.0000000
Cost after step    95: -1.0000000
Cost after step   100: -1.0000000
Optimized rotation angles: [7.15266381e-18 3.14159265e+00]


We can see that the optimization converges after approximately 40 steps.

Substituting this into the theoretical result
$\langle \psi \mid \sigma_z \mid \psi \rangle = \cos\phi_1\cos\phi_2$,
we can verify that this is indeed one possible value of the circuit
parameters that produces
$\langle \psi \mid \sigma_z \mid \psi \rangle=-1$, resulting in the
qubit being rotated to the state $\left|1\right\rangle$.

Some optimizers, such as
`~.pennylane.AdagradOptimizer`, have
internal hyperparameters that are stored in the optimizer instance.
These can be reset using the `reset`
method.
:::

Continue on to the next tutorial,
`gaussian_transformation`, to see a
similar example using continuous-variable (CV) quantum nodes.

